<a href="https://colab.research.google.com/github/parwezk/AI/blob/main/MultiAgent_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**3. Multi-Agent Collaboration via CrewAI**

CrewAI simplifies the creation of a team of autonomous agents that collaborate to achieve a goal. Each agent is assigned a specific role, goal, and backstory. They work on tasks sequentially or in parallel, with a manager agent often overseeing the process. This pattern mimics how a human team would divide and conquer a complex problem.

In [ ]:
# Install required packages
!pip install langchain langchain_community crewai langchain-google-genai tavily-python python-dotenv --quiet

In [ ]:
# Import required libraries
import os
import json
from datetime import datetime, timedelta
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew, Process

# (LLM setup is handled by CrewAI using the environment variable)

# 2. Define the Agents
travel_agent = Agent(
    role='Travel Logistics Specialist',
    goal='Find and book the most efficient flights and hotels for a business trip.',
    backstory='An expert in corporate travel, skilled at finding the best deals and routes.',
    verbose=True,
    allow_delegation=False
)

meetings_agent = Agent(
    role='Meeting Scheduling Coordinator',
    goal='Coordinate with local teams in Bangalore and Hyderabad to schedule key meetings.',
    backstory='A highly organized coordinator who manages calendars and ensures meeting slots are confirmed.',
    verbose=True,
    allow_delegation=False
)

itinerary_manager = Agent(
    role='Senior Itinerary Planner',
    goal='Integrate travel logistics and meeting schedules into a single, coherent itinerary.',
    backstory='A senior manager who oversees trip planning to ensure all objectives are met efficiently.',
    verbose=True,
    allow_delegation=True # This agent can delegate to others
)

# 3. Define the Tasks
task_details = """
Plan a 5-day business trip for a manager from Delhi.
Destinations: Bangalore (Days 1-2) and Hyderabad (Days 3-4).
Start Date: Monday, October 6th, 2025.
Purpose: Meet with local teams and attend key discussions.
"""

logistics_task = Task(
    description=f"Find and detail flight and hotel options for the trip. {task_details}",
    expected_output="A list of suggested flights (DEL-BLR, BLR-HYD, HYD-DEL) and hotel options in Bangalore and Hyderabad.",
    agent=travel_agent
)

meetings_task = Task(
    description=f"Propose a meeting schedule for Bangalore and Hyderabad. {task_details}",
    expected_output="A schedule of confirmed meeting slots with the teams in both cities for the specified dates.",
    agent=meetings_agent
)

itinerary_task = Task(
    description="Combine the flight, hotel, and meeting information into a final, consolidated, day-by-day itinerary.",
    expected_output="A complete 5-day itinerary document, including all travel details, hotel check-ins, meeting times, and buffer periods.",
    agent=itinerary_manager
)

# 4. Create and Run the Crew
trip_crew = Crew(
    agents=[travel_agent, meetings_agent, itinerary_manager],
    tasks=[logistics_task, meetings_task, itinerary_task],
    process=Process.sequential, # Tasks will be executed one after another
    verbose=True
)

final_itinerary = trip_crew.kickoff()

print("\n\n--- FINAL ITINERARY (CrewAI) ---")
print(final_itinerary)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b304f83c-a622-45a2-936b-5622cd46e784                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Logistics Specialist                                                                             │
│                                                                                                                 │
│  Task: Find and detail flight and hotel options for the trip.                                                   │
│  Plan a 5-day business trip for a manager from Delhi.                                                           │
│  Destinations: Bangalore (Days 1-2) and Hyderabad (Days 3-4).                                                   │
│  Start Date: Monday, October 6th, 2025.                                                                         │
│  Purpose: Meet with local teams and attend key discussions.                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be   │
│  set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 911464e6-4635-402c-afa3-1b076bc4255e                                                                     │
│  Agent: Travel Logistics Specialist                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: b304f83c-a622-45a2-936b-5622cd46e784                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

To use the CrewAI with OpenAI models, you need to set your OpenAI API key. If you don't have one, you can obtain it from the OpenAI website.

In Colab, you can securely store your API key using the Secrets Manager. Click on the "🔑" icon in the left sidebar, then click "Add new Secret". Name the secret `OPENAI_API_KEY` and paste your API key in the "Value" field.

Once the secret is added, run the following cell to load the API key as an environment variable.

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
!pip install crewai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 10.4 MB/s et